In [2]:
## DESCRIPTION
# meteo_temperature_passirio_boxplotBias_cells_kr11-rea
# meteo_temperature_passirio_boxplotBias_cells_kr11-rea_*

In [3]:
import sys

lib_dir = "/home/daniele/documents/github/ftt01/phd/share/lib"
sys.path.insert( 0, lib_dir )

In [4]:
from lib import *

In [5]:
wdir = "/home/daniele/documents/github/ftt01/phd/projects/era5_evaluation/"
current = DataCollector(configPath=wdir + "etc/conf/")

In [6]:
## SETUP
basin = 'passirio'
# for output name
basin_str = 'plan'

start_date_str = "2010-01-01T00:00:00"
end_date_str = "2019-12-31T23:00:00"
start_date = dt.datetime.strptime( start_date_str, '%Y-%m-%dT%H:%M:%S' )
end_date = dt.datetime.strptime( end_date_str, '%Y-%m-%dT%H:%M:%S' )

In [7]:
for el in current.sim_config:
    if el['basin'] == basin:
        ## REANALYSIS DATASET ##
        meteo_stations_rea_path = el["sim_path"] + el["meteo_path"] + "reanalysis/observations/temperature/plan/"

        temperature_df_rea = pd.DataFrame()

        t_file_tot = glob.glob( meteo_stations_rea_path + '*.csv' )

        for t_file in t_file_tot:
            data_tot = pd.read_csv(t_file,index_col=0,parse_dates=True,skiprows=4,names=[ os.path.basename(t_file)[:-4] ])
            data_tot=data_tot[start_date:end_date]
            data_tot[data_tot == -999] = None
            temperature_df_rea=pd.concat([temperature_df_rea, data_tot],axis=1)

        # spatial mean over all cells
        temperature_rea_spatial_mean = temperature_df_rea.mean(axis=1)

        # # temporal mean over all cells
        # temperature_df_rea_temporal_mean = temperature_df_rea.mean(axis=0)
        # # temporal ECDF
        # temperature_df_rea_temporal_ecdf = evaluateECDF( temperature_df_rea_temporal_mean )

        ###################################

        ## KRIGING 11X8 DATASET ##
        meteo_stations_kr11_path = el["sim_path"] + el["meteo_path"] + "kriging/observations/11x8/temperature/plan/"

        temperature_df_kr11 = pd.DataFrame()

        t_file_tot = glob.glob( meteo_stations_kr11_path + '*.csv' )

        for t_file in t_file_tot:
            data_tot = pd.read_csv(t_file,index_col=0,parse_dates=True,skiprows=4,names=[ os.path.basename(t_file)[:-4] ])
            data_tot=data_tot[start_date:end_date]
            data_tot[data_tot == -999] = None
            temperature_df_kr11=pd.concat([temperature_df_kr11, data_tot],axis=1)

        # spatial mean over all cells
        temperature_kr11_spatial_mean = temperature_df_kr11.mean(axis=1)

        # # temporal mean over all cells
        # temperature_df_kr11_temporal_mean = temperature_df_kr11.mean(axis=0)
        # # temporal ECDF
        # temperature_df_kr11_temporal_ecdf = evaluateECDF( temperature_df_kr11_temporal_mean )

In [8]:
# boxplot monthly bias
temperature_bias = temperature_rea_spatial_mean - temperature_kr11_spatial_mean
temperature_bias_monthly_sum = temperature_bias.resample('MS').mean()

output_path = current.config["output_path"] +  "meteo/" + basin_str + "/temperature/" + "/monthly/" \
    + "meteo_temperature_" + basin_str + "_monthly_spatial_mean_cells_boxplot_rea-kr11." + output_format 

createBoxPlot( temperature_bias_monthly_sum,  "Time $[month]$", "Temp. bias $[\degree C]$", \
    output_path, label="(b)", period='MS', scale_factor=0.5, output_format=output_format, my_dpi=50 )

output_path_hd = current.config["output_path"] + "meteo/" + basin_str + "/temperature/" + "/monthly/" \
    + "meteo_temperature_" + basin_str + "_monthly_spatial_mean_cells_boxplot_rea-kr11_HD." + output_format 

createBoxPlot( temperature_bias_monthly_sum,  "Time $[month]$", "Temp. bias $[\degree C]$", \
    output_path_hd, label="(b)", period='MS', scale_factor=0.5, output_format=output_format, my_dpi=600)

In [9]:
# boxplot hourly bias
temperature_bias = temperature_rea_spatial_mean - temperature_kr11_spatial_mean

output_path = current.config["output_path"] +  "meteo/" + basin_str + "/temperature/" + "/hourly/" \
    + "meteo_temperature_" + basin_str + "_hourly_spatial_mean_cells_boxplot_rea-kr11." + output_format 
createBoxPlot( temperature_bias,  "Time $[hour]$", "Temp. bias $[\degree C]$", \
    output_path, period='MS', output_format=output_format, my_dpi=50 )

output_path_hd = current.config["output_path"] + "meteo/" + basin_str + "/temperature/" + "/hourly/" \
    + "meteo_temperature_" + basin_str + "_hourly_spatial_mean_cells_boxplot_rea-kr11_HD." + output_format 
createBoxPlot( temperature_bias,  "Time $[hour]$", "Temp. bias $[\degree C]$", \
    output_path_hd, period='MS', output_format=output_format, my_dpi=600)

In [10]:
# spatial ECDF - hourly
temperature_rea_spatial_ecdf = evaluateECDF( temperature_rea_spatial_mean )
temperature_kr11_spatial_ecdf = evaluateECDF( temperature_kr11_spatial_mean )

plots = []

plt_conf = {}
plt_conf["label"] = "REA11x8"
plt_conf["color"] = '#e66101'
plots.append( (temperature_rea_spatial_ecdf, plt_conf) )

plt_conf = {}
plt_conf["label"] = "KR11x8"
plt_conf["color"] = '#5e3c99'
plots.append( (temperature_kr11_spatial_ecdf, plt_conf) )

output_file = current.config["output_path"] + "meteo/" + basin_str + "/temperature/hourly/meteo_" \
    + 'temperature_passirio_hourly_spatialECDF.' + output_format
createPlot( plots, "Temperature $[\degree C]$", "ECDF", output_file, \
    xticks=[-30,-25,-20,-15,-10,-5,0,5,10,15,20,25,30], x_rot=35, yticks=[0,10,20,30,40,50,60,70,80,90,100], label="(b)", \
    height=160, scale_factor=0.5, bbox_to_anchor=None, my_dpi=50)

output_file_hd = current.config["output_path"] + "meteo/" + basin_str + "/temperature/hourly/meteo_" \
    + 'temperature_passirio_hourly_spatialECDF_HD.' + output_format
createPlot( plots, "Temperature $[\degree C]$", "ECDF", output_file_hd, \
    xticks=[-30,-25,-20,-15,-10,-5,0,5,10,15,20,25,30], x_rot=35, yticks=[0,10,20,30,40,50,60,70,80,90,100], label="(b)", \
    height=160, scale_factor=0.5, bbox_to_anchor=None, my_dpi=600)

output_file = current.config["output_path"] + "meteo/" + basin_str + "/temperature/hourly/meteo_" \
    + 'temperature_passirio_hourly_spatialECDF_log.' + output_format
createPlot( plots, "Temperature $[\degree C]$", "ECDF", output_file, \
    xticks=[-30,-25,-20,-15,-10,-5,0,5,10,15,20,25,30], x_rot=35, yticks=[0,10,20,30,40,50,60,70,80,90,100], label="(b)", \
    height=160, scale_factor=0.5, bbox_to_anchor=None, my_dpi=50)

output_file_hd = current.config["output_path"] + "meteo/" + basin_str + "/temperature/hourly/meteo_" \
    + 'temperature_passirio_hourly_spatialECDF_log_HD.' + output_format
createPlot( plots, "Temperature $[\degree C]$", "ECDF", output_file_hd, \
    xticks=[-30,-25,-20,-15,-10,-5,0,5,10,15,20,25,30], x_rot=35, yticks=[0,10,20,30,40,50,60,70,80,90,100], label="(b)", \
    height=160, scale_factor=0.5, bbox_to_anchor=None, my_dpi=600)

In [11]:
# spatial ECDF - monthly
temperature_rea_spatial_ecdf = evaluateECDF( temperature_rea_spatial_mean.resample('MS').mean() )
temperature_kr11_spatial_ecdf = evaluateECDF( temperature_kr11_spatial_mean.resample('MS').mean() )

plots = []

plt_conf = {}
plt_conf["label"] = "REA11x8"
plt_conf["color"] = '#e66101'
plots.append( (temperature_rea_spatial_ecdf, plt_conf) )

plt_conf = {}
plt_conf["label"] = "KR11x8"
plt_conf["color"] = '#5e3c99'
plots.append( (temperature_kr11_spatial_ecdf, plt_conf) )

output_file = current.config["output_path"] + "meteo/" + basin_str + "/temperature/monthly/meteo_" \
    + 'temperature_passirio_monthly_spatialECDF.' + output_format
createPlot( plots, "Temperature $[\degree C]$", "ECDF", output_file, \
    scale_factor=0.5, my_dpi=50)

output_file_hd = current.config["output_path"] + "meteo/" + basin_str + "/temperature/monthly/meteo_" \
    + 'temperature_passirio_monthly_spatialECDF_HD.' + output_format
createPlot( plots, "Temperature $[\degree C]$", "ECDF", output_file_hd, \
    scale_factor=0.5, my_dpi=600)

output_file = current.config["output_path"] + "meteo/" + basin_str + "/temperature/monthly/meteo_" \
    + 'temperature_passirio_monthly_spatialECDF_log.' + output_format
createPlot( plots, "Temperature $[\degree C]$", "ECDF", output_file, \
    xscale='log', scale_factor=0.5, my_dpi=50)

output_file_hd = current.config["output_path"] + "meteo/" + basin_str + "/temperature/monthly/meteo_" \
    + 'temperature_passirio_monthly_spatialECDF_log_HD.' + output_format
createPlot( plots, "Temperature $[\degree C]$", "ECDF", output_file_hd, \
    xscale='log', scale_factor=0.5, my_dpi=600)

In [12]:
rea = temperature_rea_spatial_ecdf
kr = temperature_kr11_spatial_ecdf
bias_min = np.array(rea.loc[(rea['cdf'] <= 5)].index).mean() - np.array(kr.loc[(kr['cdf'] <= 5)].index).mean()
print(bias_min)

-3.2187768870509794


In [13]:
rea = temperature_rea_spatial_ecdf
kr = temperature_kr11_spatial_ecdf
bias_median = np.array(rea.loc[(rea['cdf'] >= 49.5) & (rea['cdf'] <= 50.5)].index).mean() - np.array(kr.loc[(kr['cdf'] >= 49.5) & (kr['cdf'] <= 50.5)].index).mean()
print(bias_median)

-1.0519769895833333


In [14]:
rea = temperature_rea_spatial_ecdf
kr = temperature_kr11_spatial_ecdf
bias_max = np.array(rea.loc[(rea['cdf'] >= 95)].index).mean() - np.array(kr.loc[(kr['cdf'] >= 95)].index).mean()
print(bias_max)

0.09889318959080029


In [15]:
temperature_bias = temperature_rea_spatial_mean - temperature_kr11_spatial_mean

#-------------------boxplot seasonal--------------------     
temperature_bias_jan=temperature_bias.loc[(temperature_bias.index.month==1)]
temperature_bias_feb=temperature_bias.loc[(temperature_bias.index.month==2)]
temperature_bias_mar=temperature_bias.loc[(temperature_bias.index.month==3)]
temperature_bias_apr=temperature_bias.loc[(temperature_bias.index.month==4)]
temperature_bias_may=temperature_bias.loc[(temperature_bias.index.month==5)]
temperature_bias_jun=temperature_bias.loc[(temperature_bias.index.month==6)]
temperature_bias_jul=temperature_bias.loc[(temperature_bias.index.month==7)]
temperature_bias_aug=temperature_bias.loc[(temperature_bias.index.month==8)]
temperature_bias_sep=temperature_bias.loc[(temperature_bias.index.month==9)]
temperature_bias_oct=temperature_bias.loc[(temperature_bias.index.month==10)]
temperature_bias_nov=temperature_bias.loc[(temperature_bias.index.month==11)]
temperature_bias_dec=temperature_bias.loc[(temperature_bias.index.month==12)]

temperature_bias_w=pd.concat([temperature_bias_jan,temperature_bias_feb,temperature_bias_mar])
temperature_bias_sp=pd.concat([temperature_bias_apr,temperature_bias_may,temperature_bias_jun])
temperature_bias_su=pd.concat([temperature_bias_jul,temperature_bias_aug,temperature_bias_sep])
temperature_bias_a=pd.concat([temperature_bias_oct,temperature_bias_nov,temperature_bias_dec])

output_file = current.config["output_path"] +  "meteo/" + basin_str + "/temperature/" + "/hourly/seasonal/" \
    + 'meteo_temperature_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_winter.' + output_format
createBoxPlot( temperature_bias_w, "Time $[hour]$", "Temp. bias $[\degree C]$", \
    output_file, output_format=output_format, period="H", scale_factor=0.5, my_dpi=50 )

output_file_hd = current.config["output_path"] +  "meteo/" + basin_str + "/temperature/" + "/hourly/seasonal/" \
    + 'meteo_temperature_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_winter_HD.' + output_format
createBoxPlot( temperature_bias_w, "Time $[hour]$", "Temp. bias $[\degree C]$", \
    output_file_hd, output_format=output_format, period="H", scale_factor=0.5, my_dpi=600 )

output_file = current.config["output_path"] +  "meteo/" + basin_str + "/temperature/" + "/hourly/seasonal/" \
    + 'meteo_temperature_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_spring.' + output_format
createBoxPlot( temperature_bias_sp, "Time $[hour]$", "Temp. bias $[\degree C]$", \
    output_file, output_format=output_format, period="H", scale_factor=0.5, my_dpi=50 )

output_file_hd = current.config["output_path"] +  "meteo/" + basin_str + "/temperature/" + "/hourly/seasonal/" \
    + 'meteo_temperature_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_spring_HD.' + output_format
createBoxPlot( temperature_bias_sp, "Time $[hour]$", "Temp. bias $[\degree C]$", \
    output_file_hd, output_format=output_format, period="H", scale_factor=0.5, my_dpi=600 )

output_file = current.config["output_path"] +  "meteo/" + basin_str + "/temperature/" + "/hourly/seasonal/" \
    + 'meteo_temperature_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_summer.' + output_format
createBoxPlot( temperature_bias_su, "Time $[hour]$", "Temp. bias $[\degree C]$", \
    output_file, output_format=output_format, period="H", scale_factor=0.5, my_dpi=50 )

output_file_hd = current.config["output_path"] +  "meteo/" + basin_str + "/temperature/" + "/hourly/seasonal/" \
    + 'meteo_temperature_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_summer_HD.' + output_format
createBoxPlot( temperature_bias_su, "Time $[hour]$", "Temp. bias $[\degree C]$", \
    output_file_hd, output_format=output_format, period="H", scale_factor=0.5, my_dpi=600 )

output_file = current.config["output_path"] + "meteo/" + basin_str + "/temperature/" + "/hourly/seasonal/" \
    + 'meteo_temperature_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_autumn.' + output_format
createBoxPlot( temperature_bias_a, "Time $[hour]$", "Temp. bias $[\degree C]$", \
    output_file, output_format=output_format, period="H", scale_factor=0.5, my_dpi=50 )

output_file_hd = current.config["output_path"] +  "meteo/" + basin_str + "/temperature/" + "/hourly/seasonal/" \
    + 'meteo_temperature_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_autumn_HD.' + output_format
createBoxPlot( temperature_bias_a, "Time $[hour]$", "Temp. bias $[\degree C]$", \
    output_file_hd, output_format=output_format, period="H", scale_factor=0.5, my_dpi=600 )

In [16]:
temperature_bias_monthly = temperature_bias.resample("MS").mean()

print(f"Min: {temperature_bias_monthly.min()}")
print(f"Max: {temperature_bias_monthly.max()}")
print(f"Mean: {temperature_bias_monthly.mean()}")

Min: -4.301339489919354
Max: 0.969202494623656
Mean: -1.2000030767846814


In [17]:
result_string = ""
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for i, month in enumerate(months, start=1):
    query_result = temperature_bias.loc[temperature_bias.index.month == i]
    monthly_mean = query_result.mean()  # Calculate the mean for the month
    result_string += f"Total precipitation bias for {month}: {round(monthly_mean,1)}\n"

print(result_string)

Total precipitation bias for Jan: -3.3
Total precipitation bias for Feb: -2.2
Total precipitation bias for Mar: -1.3
Total precipitation bias for Apr: -0.8
Total precipitation bias for May: -0.5
Total precipitation bias for Jun: -0.0
Total precipitation bias for Jul: 0.2
Total precipitation bias for Aug: 0.0
Total precipitation bias for Sep: -0.2
Total precipitation bias for Oct: -0.9
Total precipitation bias for Nov: -2.1
Total precipitation bias for Dec: -3.2

